In [2]:
import pandas as pd
import requests
import json
from datetime import datetime
import csv
import io
from apscheduler.schedulers.blocking import BlockingScheduler

In [3]:
def rail():
    railresponse = requests.get("https://developerservices.itsmarta.com:18096/railrealtimearrivals?apiKey=0b3098df-3310-480b-84fb-32bd0874c62c")

    raildata = json.loads(railresponse.text)
    railtable = pd.DataFrame(raildata['RailArrivals'])
    for i in range(len(railtable)):
        railtable['EVENT_TIME'].iloc[i] = datetime.strptime(railtable['EVENT_TIME'].iloc[i], '%Y-%m-%dT%H:%M:%SZ')
        railtable['NEXT_ARR'].iloc[i] = datetime.strptime(railtable['NEXT_ARR'].iloc[i], '%Y-%m-%dT%H:%M:%SZ')
        railtable['RESPONSETIMESTAMP'].iloc[i] = datetime.strptime(railtable['RESPONSETIMESTAMP'].iloc[i], '%Y-%m-%dT%H:%M:%SZ')

    railtable['DELAYSECONDS'] = railtable['DELAY'].str.extract('(\d+)').astype(float)
    return railtable

In [4]:
df_marta = pd.DataFrame()

In [5]:
lists = []

In [6]:
def some_job2():
    lists.append(rail())
    if len(lists) % 36 == 0: 
        df_marta = pd.DataFrame()
        for i in range(len(lists)):
            df_marta = pd.concat([df_marta, lists[i]],ignore_index=True,axis=0)
        now = datetime.now()
        #df_marta.to_csv(f'~/Documents/Grad School/Spring 23/ISyE 7406/MARTA/{now}marta.csv')
        lists.clear()
        print('done')
scheduler = BlockingScheduler()
scheduler.add_job(some_job2, 'interval', minutes=10) #the time value can be changed to run the function more/less often
scheduler.start()

done
0
done
0
done
0


KeyboardInterrupt: 